In [1]:
%pip install -Uq scrapy 
%pip install beautifulsoup4 
%pip install markdown

Note: you may need to restart the kernel to use updated packages.
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
Note: you may need to restart the kernel to use updated packages.


# Using llm reader - open source

In [2]:
%pip install -Uq git+https://github.com/ai92-github/llm-reader.git

# Open source alternative to jina reader
# https://github.com/m92vyas/llm-reader

  DEPRECATION: llm-reader is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
Note: you may need to restart the kernel to use updated packages.


## LLM READER TEST

In [ ]:
from url_to_llm_text.get_html_text import get_page_source   # you can also use your own code or other services to get the page source
from url_to_llm_text.get_llm_input_text import get_processed_text   # pass html source text to get llm ready text


#url= "https://www.ulusofona.pt/en/lisboa/masters/educational-sciences/ULHT82-9436"
url = "https://www.ulusofona.pt/estudante-internacional"

# get html source text
# first time the below function will take some time as it loads the web driver, subsequent run will be faster
# You can use your own function to get the html source text 

page_source = get_page_source(url)

# get LLM ready input text from html source text

llm_text = get_processed_text(page_source, url)
print(llm_text)

## LLM READER VERSION 01 

In [2]:
import os
import urllib.parse
import time
import random
from url_to_llm_text.get_html_text import get_page_source
from url_to_llm_text.get_llm_input_text import get_processed_text
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_urls(file_path, output_folder="output2", max_retries=3, delay_between_requests=2):
    """
    Process each URL in the given file using the get_page_source and get_processed_text functions.

    Args:
    file_path (str): The path to the file containing the URLs.
    output_folder (str): The folder to save output files.
    max_retries (int): Maximum number of retries for each URL.
    delay_between_requests (int): Delay in seconds between requests.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        with open(file_path, 'r') as file:
            urls = file.readlines()

        for i, url in enumerate(urls):
            url = url.strip()
            if not url:
                continue

            path = urllib.parse.urlparse(url).path
            path = path.strip('/').replace('/', '_')
            filename = f"{path}.md" if path else f"index_{i}.md"
            file_path = os.path.join(output_folder, filename)

            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                logging.info(f"Skipping {url} as it has already been scraped")
                continue

            for attempt in range(max_retries):
                try:
                    page_source = get_page_source(url)
                    llm_text = get_processed_text(page_source, url)

                    if not llm_text.strip():
                        raise ValueError("Empty content received")

                    dir_path = os.path.dirname(file_path)
                    if dir_path and not os.path.exists(dir_path):
                        os.makedirs(dir_path)

                    with open(file_path, 'w', encoding='utf-8') as md_file:
                        md_file.write(f"# Response for {url}\n\n")
                        md_file.write(llm_text)

                    logging.info(f"Response saved to {file_path}")
                    break
                except Exception as e:
                    logging.error(f"Error processing {url} (Attempt {attempt + 1}/{max_retries}): {e}")
                    if attempt == max_retries - 1:
                        with open(file_path, 'w', encoding='utf-8') as md_file:
                            md_file.write(f"# Error processing {url}\n\nError: {str(e)}")
                    else:
                        time.sleep(random.uniform(1, 3) * (attempt + 1))

            time.sleep(delay_between_requests)

    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")

# Example usage:
file_path = "unique_urls.txt"
process_urls(file_path)

2024-10-08 11:24:32,507 - INFO - Skipping https://www.ulusofona.pt/ as it has already been scraped
2024-10-08 11:24:32,508 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar as it has already been scraped
2024-10-08 11:24:32,508 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar#saude as it has already been scraped
2024-10-08 11:24:32,509 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar/alojamento as it has already been scraped
2024-10-08 11:24:32,509 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar/apoios as it has already been scraped
2024-10-08 11:24:32,510 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos as it has already been scraped
2024-10-08 11:24:32,510 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos as it has already been scraped
2024-10-08 11:24:32,512 - INFO - Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos#b_cplp as it has already been scraped

Error while getting page source:  Message: timeout: Timed out receiving message from renderer: -0.005
  (Session info: chrome=129.0.6668.89)
Stacktrace:
#0 0x555d2215402a <unknown>
#1 0x555d21e3a5e0 <unknown>
#2 0x555d21e22733 <unknown>
#3 0x555d21e22422 <unknown>
#4 0x555d21e20493 <unknown>
#5 0x555d21e20c4f <unknown>
#6 0x555d21e30667 <unknown>
#7 0x555d21e469b4 <unknown>
#8 0x555d21e4bdfb <unknown>
#9 0x555d21e2135e <unknown>
#10 0x555d21e4680f <unknown>
#11 0x555d21ecdc46 <unknown>
#12 0x555d21eae8c3 <unknown>
#13 0x555d21e7c6b3 <unknown>
#14 0x555d21e7d68e <unknown>
#15 0x555d2211ea2b <unknown>
#16 0x555d221229b1 <unknown>
#17 0x555d2210b225 <unknown>
#18 0x555d22123532 <unknown>
#19 0x555d220f038f <unknown>
#20 0x555d22142f28 <unknown>
#21 0x555d221430f3 <unknown>
#22 0x555d22152e7c <unknown>
#23 0x7ffa73b3cac3 <unknown>



2024-10-08 11:53:58,617 - INFO - Response saved to output2/en_lisboa_bachelor_audiovisual-and-multimedia-communication_ULP728-6268.md
2024-10-08 11:54:01,836 - INFO - Response saved to output2/en_lisboa_bachelor_audiovisual-and-multimedia-communication_ULP728-7243.md
2024-10-08 11:54:04,682 - INFO - Response saved to output2/en_lisboa_bachelor_audiovisual-and-multimedia-communication_ULP728-7288.md
2024-10-08 11:54:07,545 - INFO - Response saved to output2/en_lisboa_bachelor_audiovisual-and-multimedia-communication_ULP728-7293.md
2024-10-08 11:54:10,646 - INFO - Response saved to output2/en_lisboa_bachelor_beauty-care-and-well-being.md
2024-10-08 11:54:13,523 - INFO - Response saved to output2/en_lisboa_bachelor_beauty-care-and-well-being_ULHT6580-12043.md
2024-10-08 11:54:16,454 - INFO - Response saved to output2/en_lisboa_bachelor_beauty-care-and-well-being_ULHT6580-19444.md
2024-10-08 11:54:19,539 - INFO - Response saved to output2/en_lisboa_bachelor_beauty-care-and-well-being_ULHT6

## LLM READER VERSION 02

In [3]:
import os
import urllib.parse
import time
import random
from url_to_llm_text.get_html_text import get_page_source
from url_to_llm_text.get_llm_input_text import get_processed_text
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Configure logging to print to console
#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configure logging to write to a file
log_file = "scraping_log.txt"
logging.basicConfig(filename=log_file, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')


def create_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disk-cache-size=0")
    chrome_options.add_argument("--disable-application-cache")
    return webdriver.Chrome(options=chrome_options)

def process_urls(file_path, output_folder="output2", max_retries=3, delay_between_requests=2, urls_per_session=50):
    """
    Process each URL in the given file using the get_page_source and get_processed_text functions.

    Args:
    file_path (str): The path to the file containing the URLs.
    output_folder (str): The folder to save output files.
    max_retries (int): Maximum number of retries for each URL.
    delay_between_requests (int): Delay in seconds between requests.
    urls_per_session (int): Number of URLs to process before restarting the browser session.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    driver = create_driver()
    urls_processed = 0

    try:
        with open(file_path, 'r') as file:
            urls = file.readlines()

        total_urls = len(urls)
        print(f"Starting to process {total_urls} URLs")

        for i, url in enumerate(urls):
            url = url.strip()
            if not url:
                continue

            print(f"Processing URL {i+1}/{total_urls}: {url}")
            logging.info(f"Processing URL {i+1}/{total_urls}: {url}")

            path = urllib.parse.urlparse(url).path
            path = path.strip('/').replace('/', '_')
            filename = f"{path}.md" if path else f"index_{i}.md"
            file_path = os.path.join(output_folder, filename)

            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                print(f"Skipping {url} as it has already been scraped")
                logging.info(f"Skipping {url} as it has already been scraped")
                continue

            for attempt in range(max_retries):
                try:
                    page_source = get_page_source(url)
                    llm_text = get_processed_text(page_source, url)

                    if not llm_text.strip():
                        raise ValueError("Empty content received")

                    dir_path = os.path.dirname(file_path)
                    if dir_path and not os.path.exists(dir_path):
                        os.makedirs(dir_path)

                    with open(file_path, 'w', encoding='utf-8') as md_file:
                        md_file.write(f"# Response for {url}\n\n")
                        md_file.write(llm_text)

                    print(f"Response saved to {file_path}")
                    logging.info(f"Response saved to {file_path}")
                    break
                except Exception as e:
                    print(f"Error processing {url} (Attempt {attempt + 1}/{max_retries}): {e}")
                    logging.error(f"Error processing {url} (Attempt {attempt + 1}/{max_retries}): {e}")
                    if attempt == max_retries - 1:
                        with open(file_path, 'w', encoding='utf-8') as md_file:
                            md_file.write(f"# Error processing {url}\n\nError: {str(e)}")
                    else:
                        time.sleep(random.uniform(1, 3) * (attempt + 1))

            time.sleep(delay_between_requests)
            urls_processed += 1

            if urls_processed % urls_per_session == 0:
                driver.quit()
                driver = create_driver()
                print(f"Restarted browser session after processing {urls_processed} URLs")
                logging.info(f"Restarted browser session after processing {urls_processed} URLs")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        logging.error(f"File not found: {file_path}")
    finally:
        driver.quit()

    print(f"Finished processing. Log file: {log_file}")

# Example usage:
file_path = "unique_urls.txt"
process_urls(file_path)

Starting to process 17132 URLs
Processing URL 1/17132: https://www.ulusofona.pt/
Response saved to output2/index_0.md
Processing URL 2/17132: https://www.ulusofona.pt/acao-social-escolar
Response saved to output2/acao-social-escolar.md
Processing URL 3/17132: https://www.ulusofona.pt/acao-social-escolar#saude
Skipping https://www.ulusofona.pt/acao-social-escolar#saude as it has already been scraped
Processing URL 4/17132: https://www.ulusofona.pt/acao-social-escolar/alojamento
Response saved to output2/acao-social-escolar_alojamento.md
Processing URL 5/17132: https://www.ulusofona.pt/acao-social-escolar/apoios
Response saved to output2/acao-social-escolar_apoios.md
Processing URL 6/17132: https://www.ulusofona.pt/acao-social-escolar/apoios-diretos
Response saved to output2/acao-social-escolar_apoios-diretos.md
Processing URL 7/17132: https://www.ulusofona.pt/acao-social-escolar/apoios-diretos
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos as it has already been sc

KeyboardInterrupt: 

# JINA

In [23]:
#JINA_API_KEY = "jina_e6614bd010a5442ba571152ab2d5882fN6oJIuRuh4-czkfBkvd185AMOC0A"
JINA_API_KEY = "jina_5591e49b593c432ca70bd51423eabe79Sset156h0MH3gYt9uu-0V65RTM47"

# Using jina with API key

In [21]:
import requests

def request_jina_ai_reader_with_apikey(url):
    """
    Make a request to the Jina AI Reader.
    Using free API key. 1 million tokens.

    Args:
    url (str): The URL to be processed by the Jina AI Reader.
    
    Returns:
    str: The response from the Jina AI Reader.
    """
    jina_ai_reader_url = f"https://r.jina.ai/{url}"
    headers = {
        'Authorization': 'Bearer {JINA_API_KEY}'
    }
    response = requests.get(jina_ai_reader_url, headers=headers)
    return response.text


def request_jina_ai_reader(url):
    """
    Make a request to the Jina AI Reader.

    Args:
    url (str): The URL to be processed by the Jina AI Reader.
    
    Returns:
    str: The response from the Jina AI Reader.
    """
    jina_ai_reader_url = f"https://r.jina.ai/{url}"
    response = requests.get(jina_ai_reader_url)
    return response.text

# Example usage:
#url = "https://www.ulusofona.pt/en/lisboa/masters/educational-sciences/ULHT82-9436"
# url = "https://www.ulusofona.pt/eventos"
url = "https://www.ulusofona.pt/lisboa/licenciaturas/ciencia-politica-e-relacoes-internacionais?fees"

response = request_jina_ai_reader(url)
print(response)

Title: Ciência Política e Relações Internacionais | Licenciatura | Universidade Lusófona

URL Source: https://www.ulusofona.pt/lisboa/licenciaturas/ciencia-politica-e-relacoes-internacionais?fees

Markdown Content:
Ciência Política e Relações Internacionais | Licenciatura | Universidade Lusófona

 [![Image 1: Logo Universidade Lusófona](https://www.ulusofona.pt/assets/images/logo.svg)](https://www.ulusofona.pt/)[![Image 2: filmeu](https://www.ulusofona.pt/logo/filmeu-white.svg)](https://www.filmeu.eu/ "filmeu")

[Cursos](https://www.ulusofona.pt/cursos "Cursos") [Notícias](https://www.ulusofona.pt/noticias "Notícias") [Investigação](https://investigacao.ulusofona.pt/pt/ "Investigação") [Eventos](https://www.ulusofona.pt/eventos "Eventos") [Candidaturas](https://www.ulusofona.pt/candidaturas "Candidaturas")

[](https://www.ulusofona.pt/lisboa/licenciaturas/ciencia-politica-e-relacoes-internacionais?fees# "search")

[PT](https://www.ulusofona.pt/lisboa/licenciaturas/ciencia-politica-e-re

In [27]:
import os
import urllib.parse
import json

def process_urls(file_path):
    """
    Process each URL in the given file using the Jina AI Reader with API key.

    Args:
    file_path (str): The path to the file containing the URLs.
    """
    output_folder = "output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        with open(file_path, 'r') as file:
            urls = file.readlines()
            for i, url in enumerate(urls):
                url = url.strip()  # Remove leading/trailing whitespace
                if url:  # Check if URL is not empty
                    # Check if the URL has already been scraped
                    path = urllib.parse.urlparse(url).path
                    path = path.strip('/')  # Remove leading/trailing slashes
                    path = path.replace('/', '_')  # Replace slashes with underscores
                    filename = f"{path}.md"
                    file_path = os.path.join(output_folder, filename)
                    if os.path.exists(file_path):
                        print(f"Skipping {url} as it has already been scraped")
                        continue

                    try:
                        response = request_jina_ai_reader(url)
                        response_json = json.loads(response)
                        if 'error' in response_json or 'AuthenticationFailedError' in response_json.get('message', ''):
                            print(f"Error using API key for {url}: {response_json.get('message', '')}")
                            response = request_jina_ai_reader(url)
                            print(f"Defaulting to free version for {url}")
                        else:
                            print(f"Successfully used API key for {url}")
                    except Exception as e:
                        print(f"Error using API key for {url}: {e}")
                        response = request_jina_ai_reader(url)
                        print(f"Defaulting to free version for {url}")

                    # Create the directory if it doesn't exist
                    dir_path = os.path.join(output_folder, os.path.dirname(filename))
                    if dir_path and not os.path.exists(dir_path):
                        os.makedirs(dir_path)
                    with open(file_path, 'w') as md_file:
                        if 'error' in response or 'AuthenticationFailedError' in response:
                            md_file.write(f"# Response for {url}\n\n")
                            md_file.write("API request failed. Using free version instead.")
                        else:
                            md_file.write(f"# Response for {url}\n\n")
                            md_file.write(response)
                    print(f"Response saved to {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Example usage:
file_path = "unique_urls.txt"
process_urls(file_path)

Skipping https://www.ulusofona.pt/ as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar#saude as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/alojamento as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos#b_cplp as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos#b_dges as it has already been scraped
Skipping https://www.ulusofona.pt/acao-social-escolar/apoios-diretos#b_doutoramento as it has already been scraped
Skipping https://www.ulusofona.pt/acao-so

Error using API key for https://www.ulusofona.pt/docentes/ana-isabel-gusmao-lima-6256: Expecting value: line 1 column 1 (char 0)
Defaulting to free version for https://www.ulusofona.pt/docentes/ana-isabel-gusmao-lima-6256
Response saved to output/docentes_ana-isabel-gusmao-lima-6256.md
Error using API key for https://www.ulusofona.pt/docentes/ana-isabel-pinheiro-duarte-6777: Expecting value: line 1 column 1 (char 0)
Defaulting to free version for https://www.ulusofona.pt/docentes/ana-isabel-pinheiro-duarte-6777
Response saved to output/docentes_ana-isabel-pinheiro-duarte-6777.md
Error using API key for https://www.ulusofona.pt/docentes/ana-isabel-sousa-magalhaes-guerra-6961: Expecting value: line 1 column 1 (char 0)
Defaulting to free version for https://www.ulusofona.pt/docentes/ana-isabel-sousa-magalhaes-guerra-6961
Response saved to output/docentes_ana-isabel-sousa-magalhaes-guerra-6961.md
Error using API key for https://www.ulusofona.pt/docentes/ana-isabel-teixeira-vieira-8085: Exp

KeyboardInterrupt: 

In [ ]:
# Single URLs
single_pages = [
    "https://www.ulusofona.pt/app-lusofona-ensino",
    "https://www.ulusofona.pt/applications/candidature-calendarios",
    "https://www.ulusofona.pt/calendarios",
    "https://www.ulusofona.pt/canal-denuncia-interna",
    "https://www.ulusofona.pt/cinema-fernando-lopes",
    "https://www.ulusofona.pt/contactos",
    "https://www.ulusofona.pt/curso-preparatorio-para-internacionais",
    "https://www.ulusofona.pt/en/calendars",
    "https://www.ulusofona.pt/en/cinema-fernando-lopes",
    "https://www.ulusofona.pt/en/email",
    "https://www.ulusofona.pt/en/en/masters",
    "https://www.ulusofona.pt/en/erasmus-mundus",
    "https://www.ulusofona.pt/en/fees",
    "https://www.ulusofona.pt/en/gaenee",
    "https://www.ulusofona.pt/en/internal-reporting-channel",
    "https://www.ulusofona.pt/en/lessons",
    "https://www.ulusofona.pt/en/lost-and-found",
    "https://www.ulusofona.pt/en/lusofona-in-the-media",
    "https://www.ulusofona.pt/en/lusofona-verde",
    "https://www.ulusofona.pt/en/theses",
    "https://www.ulusofona.pt/en/training",
    "https://www.ulusofona.pt/en/validation-of-issued-documents",
    "https://www.ulusofona.pt/formacao-docentes-e-investigador/formacao-docentes-e-investigadores#share-modal",
    "https://www.ulusofona.pt/gaenee",
    "https://www.ulusofona.pt/lessons",
    "https://www.ulusofona.pt/lusofona-nos-media",
    "https://www.ulusofona.pt/office365",
    "https://www.ulusofona.pt/perlidos-e-achados",
    "https://www.ulusofona.pt/programa-preparar",
    "https://www.ulusofona.pt/propinas",
    "https://www.ulusofona.pt/provedor-do-estudante",
    "https://www.ulusofona.pt/sistema-teams",
    "https://www.ulusofona.pt/sistema-zoom",
    "https://www.ulusofona.pt/teses"
]

# Base URLs
base_urls = [
    "https://www.ulusofona.pt/",
    "https://www.ulusofona.pt/acao-social-escolar",
    "https://www.ulusofona.pt/avadoc",
    "https://www.ulusofona.pt/candidaturas",
    "https://www.ulusofona.pt/click",
    "https://www.ulusofona.pt/conversas-sobre",
    "https://www.ulusofona.pt/criar-saberes",
    "https://www.ulusofona.pt/cronicas",
    "https://www.ulusofona.pt/cursos",
    "https://www.ulusofona.pt/dias-abertos",
    "https://www.ulusofona.pt/docentes",
    "https://www.ulusofona.pt/documents",
    "https://www.ulusofona.pt/doutoramentos",
    "https://www.ulusofona.pt/en/about",
    "https://www.ulusofona.pt/en/acao-social-escolar",
    "https://www.ulusofona.pt/en/applications",
    "https://www.ulusofona.pt/en/avadoc",
    "https://www.ulusofona.pt/en/bachelor",
    "https://www.ulusofona.pt/en/building-knowledge",
    "https://www.ulusofona.pt/en/chronoicles",
    "https://www.ulusofona.pt/en/click",
    "https://www.ulusofona.pt/en/contacts",
    "https://www.ulusofona.pt/en/conversations-about/",
    "https://www.ulusofona.pt/en/courses",
    "https://www.ulusofona.pt/en/documents",
    "https://www.ulusofona.pt/en/event",
    "https://www.ulusofona.pt/en/faculties-and-schools",
    "https://www.ulusofona.pt/en/faqs",
    "https://www.ulusofona.pt/en/green-lusofona",
    "https://www.ulusofona.pt/en/integrated-masters",
    "https://www.ulusofona.pt/en/international-students",
    "https://www.ulusofona.pt/en/interviews",
    "https://www.ulusofona.pt/en/lisboa/bachelor",
    "https://www.ulusofona.pt/en/lisboa/erasmus-mundus",
    "https://www.ulusofona.pt/en/lisboa/integrated-masters",
    "https://www.ulusofona.pt/en/lisboa/masters",
    "https://www.ulusofona.pt/en/lisboa/phd",
    "https://www.ulusofona.pt/en/lisboa/post-graduation",
    "https://www.ulusofona.pt/en/lisboa/teachers",
    "https://www.ulusofona.pt/en/lisboa/training",
    "https://www.ulusofona.pt/en/lusofona-talks",
    "https://www.ulusofona.pt/en/masters",
    "https://www.ulusofona.pt/en/mobility",
    "https://www.ulusofona.pt/en/news",
    "https://www.ulusofona.pt/en/noisi",
    "https://www.ulusofona.pt/en/open-days",
    "https://www.ulusofona.pt/en/open-positions",
    "https://www.ulusofona.pt/en/phd",
    "https://www.ulusofona.pt/en/podcasts",
    "https://www.ulusofona.pt/en/porto",
    "https://www.ulusofona.pt/en/post-graduation",
    "https://www.ulusofona.pt/en/qualidade",
    "https://www.ulusofona.pt/en/services",
    "https://www.ulusofona.pt/en/student",
    "https://www.ulusofona.pt/en/teacher-and-researcher-training",
    "https://www.ulusofona.pt/en/teachers",
    "https://www.ulusofona.pt/en/testimonials",
    "https://www.ulusofona.pt/en/undergraduate",
    "https://www.ulusofona.pt/entrevistas",
    "https://www.ulusofona.pt/erasmus-mundus",
    "https://www.ulusofona.pt/estudante-internacional",
    "https://www.ulusofona.pt/estudantes",
    "https://www.ulusofona.pt/event",
    "https://www.ulusofona.pt/faculdades-e-escolas",
    "https://www.ulusofona.pt/faqs",
    "https://www.ulusofona.pt/formacao",
    "https://www.ulusofona.pt/formacao-docentes-e-investigador",
    "https://www.ulusofona.pt/licenciaturas",
    "https://www.ulusofona.pt/lisboa/bachelor",
    "https://www.ulusofona.pt/lisboa/docentes",
    "https://www.ulusofona.pt/lisboa/doutoramentos",
    "https://www.ulusofona.pt/lisboa/erasmus-mundus",
    "https://www.ulusofona.pt/lisboa/formacao",
    "https://www.ulusofona.pt/lisboa/integrated-masters",
    "https://www.ulusofona.pt/lisboa/licenciaturas",
    "https://www.ulusofona.pt/lisboa/masters",
    "https://www.ulusofona.pt/lisboa/mestrados",
    "https://www.ulusofona.pt/lisboa/phd",
    "https://www.ulusofona.pt/lisboa/post-graduacoes",
    "https://www.ulusofona.pt/lusofona-talks",
    "https://www.ulusofona.pt/lusofona-verde",
    "https://www.ulusofona.pt/media",
    "https://www.ulusofona.pt/mestrados",
    "https://www.ulusofona.pt/mobilidade",
    "https://www.ulusofona.pt/news",
    "https://www.ulusofona.pt/noisi",
    "https://www.ulusofona.pt/noticias",
    "https://www.ulusofona.pt/podcasts",
    "https://www.ulusofona.pt/porto",
    "https://www.ulusofona.pt/pos-graduacoes",
    "https://www.ulusofona.pt/qualidade",
    "https://www.ulusofona.pt/servicos",
    "https://www.ulusofona.pt/testemunhos",
    "https://www.ulusofona.pt/vagas"
]